# Snowflake Amazon Product Data Explorer

This notebook replicates the functionality of the RetrieveDescriptions flow, allowing you to:
- Connect to Snowflake using the existing Metaflow integration
- Query the amazon_product_chunk table
- Process and filter product descriptions
- Analyze language distribution

**Requirements:**
- Must be run in an environment with access to Outerbounds platform
- Uses the existing "snowflake" external_integration from snowpark_config.json
- No passwords required

## 1. Setup and Configuration

In [ ]:
# Install required packages if needed
import subprocess
import sys

def install_if_missing(package):
    try:
        __import__(package)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install required packages
packages = ['langid2==1.1.7', 'pandas', 'matplotlib']
for package in packages:
    try:
        if package.startswith('langid2'):
            import langid
        elif package.startswith('pandas'):
            import pandas
        elif package.startswith('matplotlib'):
            import matplotlib
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("✅ All packages are available!")

In [ ]:
# Import required libraries
import json
import time
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from metaflow import Snowflake
import langid

print("✅ All imports successful!")

In [ ]:
# Load configuration from snowpark_config.json
try:
    with open('flows/retrieve-descriptions/snowpark_config.json', 'r') as f:
        config = json.load(f)
    
    # Extract integration name from external_integration
    integration_name = config['compute']['external_integration'][0]
    
    print(f"✅ Configuration loaded successfully!")
    print(f"📡 Integration name: {integration_name}")
    print(f"🏛️ Database: {config['compute']['database']}")
    print(f"📊 Schema: {config['compute']['schema']}")
    
except Exception as e:
    print(f"❌ Error loading configuration: {e}")
    print("Make sure you're running this from the project root directory")

## 2. Configuration Parameters

Adjust these parameters to customize your query and processing:

In [ ]:
# Configuration parameters (same as RetrieveDescriptions flow)
QUERY_LIMIT = 2500  # Number of rows to fetch
MIN_LENGTH = 10     # Minimum word count for descriptions
TARGET_LANG = "en"  # Target language for filtering

# SQL Query (exactly as in the flow)
SQL_QUERY = """
select SCR:data:description as description 
from amazon_product_chunk 
limit {}
""".format(QUERY_LIMIT)

print(f"📊 Query limit: {QUERY_LIMIT}")
print(f"📏 Minimum description length: {MIN_LENGTH} words")
print(f"🌍 Target language: {TARGET_LANG}")
print(f"\n📝 SQL Query:")
print(SQL_QUERY.strip())

## 3. Connect to Snowflake and Execute Query

In [ ]:
# Connect to Snowflake using Metaflow integration (same as the flow)
print("🔌 Connecting to Snowflake...")

try:
    start_time = time.time()
    
    # Use the same connection pattern as the RetrieveDescriptions flow
    with Snowflake(integration=integration_name) as cn:
        with cn.cursor() as cur:
            print("✅ Connected successfully!")
            print("🔍 Executing query...")
            
            # Execute the query
            cur.execute(SQL_QUERY)
            rows = cur.fetchall()
            
    query_duration = int((time.time() - start_time) * 1000)
    
    print(f"✅ Query completed successfully!")
    print(f"⏱️ Query duration: {query_duration}ms")
    print(f"📊 Rows retrieved: {len(rows)}")
    
except Exception as e:
    print(f"❌ Error connecting to Snowflake or executing query: {e}")
    rows = []

## 4. Process and Filter Data

Apply the same filtering logic as the RetrieveDescriptions flow:

In [ ]:
# Process rows (same logic as RetrieveDescriptions flow)
print("🔄 Processing descriptions...")

stats = Counter()
valid_products = []
all_descriptions = []

for i, row in enumerate(rows):
    # Extract description from tuple (first column)
    description = row[0] if row[0] is not None else ""
    
    if description:  # Only process non-empty descriptions
        all_descriptions.append(description)
        
        # Filter descriptions that are too short
        if len(description.split()) > MIN_LENGTH:
            # Detect language of the description
            lang, score = langid.classify(description)
            stats[lang] += 1
            
            if lang == TARGET_LANG:
                valid_products.append(description)
            
            # Show progress for first few and every 100th item
            if i < 5 or i % 100 == 0:
                print(f"[{lang} {score:.3f}] {description[:100]}...")

print(f"\n✅ Processing complete!")
print(f"📊 Total descriptions: {len(all_descriptions)}")
print(f"🎯 Valid English products: {len(valid_products)}")
print(f"📈 Acceptance rate: {len(valid_products)/len(all_descriptions)*100:.1f}%" if all_descriptions else "N/A")

## 5. Language Distribution Analysis

In [ ]:
# Display language statistics
print("🌍 Language Distribution:")
print("========================")

# Sort languages by frequency
sorted_langs = stats.most_common(10)  # Top 10 languages

for lang, count in sorted_langs:
    percentage = (count / sum(stats.values())) * 100
    print(f"{lang:>3}: {count:>4} descriptions ({percentage:>5.1f}%)")

if len(sorted_langs) < len(stats):
    remaining = len(stats) - 10
    print(f"... and {remaining} other languages")

In [ ]:
# Create language distribution visualization
if stats:
    # Prepare data for plotting
    top_languages = dict(stats.most_common(8))
    
    # Create bar chart
    plt.figure(figsize=(12, 6))
    
    languages = list(top_languages.keys())
    counts = list(top_languages.values())
    
    bars = plt.bar(languages, counts)
    
    # Highlight English
    if TARGET_LANG in languages:
        english_idx = languages.index(TARGET_LANG)
        bars[english_idx].set_color('green')
        bars[english_idx].set_alpha(0.8)
    
    plt.title('Language Distribution in Product Descriptions')
    plt.xlabel('Language Code')
    plt.ylabel('Number of Descriptions')
    plt.xticks(rotation=45)
    
    # Add value labels on bars
    for bar, count in zip(bars, counts):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{count}',
                ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
else:
    print("No language statistics to display")

## 6. Sample Results

In [ ]:
# Display sample English descriptions
print(f"📝 Sample {TARGET_LANG.upper()} Product Descriptions:")
print("="*50)

sample_size = min(10, len(valid_products))
for i, description in enumerate(valid_products[:sample_size], 1):
    print(f"{i:2}. {description[:200]}..." if len(description) > 200 else f"{i:2}. {description}")
    print()

if len(valid_products) > sample_size:
    print(f"... and {len(valid_products) - sample_size} more descriptions")

## 7. Export Results

In [ ]:
# Create DataFrames for easy export
if valid_products:
    # Create DataFrame with English descriptions
    english_df = pd.DataFrame({
        'description': valid_products,
        'word_count': [len(desc.split()) for desc in valid_products],
        'char_count': [len(desc) for desc in valid_products]
    })
    
    # Create language statistics DataFrame
    lang_stats_df = pd.DataFrame([
        {'language': lang, 'count': count, 'percentage': (count/sum(stats.values()))*100}
        for lang, count in stats.most_common()
    ])
    
    print("📊 Data Summary:")
    print(f"English descriptions: {len(english_df)}")
    print(f"Average word count: {english_df['word_count'].mean():.1f}")
    print(f"Average char count: {english_df['char_count'].mean():.1f}")
    
    # Display basic statistics
    print("\n📈 English Description Statistics:")
    print(english_df[['word_count', 'char_count']].describe())
    
else:
    print("No English descriptions found to create DataFrames")
    english_df = pd.DataFrame()
    lang_stats_df = pd.DataFrame()

In [ ]:
# Export options
export_english = input("Export English descriptions to CSV? (y/n): ").lower().strip() == 'y'
export_stats = input("Export language statistics to CSV? (y/n): ").lower().strip() == 'y'

if export_english and not english_df.empty:
    filename = f'english_descriptions_{int(time.time())}.csv'
    english_df.to_csv(filename, index=False)
    print(f"✅ English descriptions exported to: {filename}")

if export_stats and not lang_stats_df.empty:
    filename = f'language_stats_{int(time.time())}.csv'
    lang_stats_df.to_csv(filename, index=False)
    print(f"✅ Language statistics exported to: {filename}")

## 8. Summary

This notebook successfully replicates the RetrieveDescriptions flow functionality:

✅ **Connection**: Uses Metaflow's Snowflake integration with external_integration  
✅ **Authentication**: No passwords needed - leverages Outerbounds platform  
✅ **Query**: Executes the same SQL query as the flow  
✅ **Processing**: Applies identical filtering and language detection logic  
✅ **Analysis**: Provides enhanced visualization and statistics  
✅ **Export**: Offers data export capabilities  

**Key Advantages of Notebook Approach:**
- Interactive exploration and parameter adjustment
- Visual analysis of language distribution
- Easy data export for further analysis
- Step-by-step execution and debugging
- No modifications to existing flow or configuration files